# Buy Now Pay Later Project Summary
## Industrial Project Group 9


## Table of Contents
1. [ETL Pipeline](##ETL-Pipeline)
2. [Preliminary Analysis](##Preliminary-Analysis)
3. [Geospatial Analysis](##Geospatial-Analysis)
4. [Visualisation](##Visualisation)
5. [Consumer Fraud Probability Model](##Consumer-Fraud-Probability-Model)
6. [Merchant Fraud Probability Model](##Merchant-Fraud-Probability-Model)
7. [Ranking System](##Ranking-System)
8. [Segmenting Merchants](#Segmenting-Merchants)


## ETL Pipeline
The main data was provided by the BNPL firm, so we didn’t use any APIs or Python libraries to retrieve it. For external datasets, we used `urlretrieve` to download data from the **Australian Bureau of Statistics**, while postcode-to-LGA mapping data had to be downloaded manually from the [website](https://www.matthewproctor.com/Content/postcodes/australian_postcodes.csv).

### ABS's Dataset 
We used two datasets from the ABS: **Personal Income in Australia** and **Personal Fraud**. The first dataset contains information on the median and mean income, as well as the median age of earners in each LGA region. The second dataset reports the percentage of personal fraud, including card fraud, identity theft, and scams, for each state.

In the first dataset, some LGA codes in Western Australia lacked entries for median income, mean income, and median age. We imputed these missing values using the respective state's averages. We also found a discrepancy in the total number of earners. Western Australia’s total is listed as *1,585,093*, but summing the earners across all LGAs gives *1,581,061*, a difference of *32*. While the cause is unclear, we split this difference between two missing LGAs. Although not a perfect solution, we believe imputing these small numbers will not significantly impact the fraud probability prediction, especially since we cannot confirm whether consumers reside in these LGAs.

For the second dataset, minimal preprocessing was required aside from renaming columns.

### Postcode-LGA Mapping Data

For this dataset, we selected essential columns for mapping. When a postcode was missing an LGA code, we found the nearest neighboring postcode using coordinates. If the neighbor had a valid LGA code, we assigned it to the missing one. We used a simple K-Nearest Neighbour with `k=1` for this task.

Next, we merged this dataset with ABS data. After the merge, two postcodes from the mapping data were not present in the income data, resulting in null values. We applied the same approach to impute the missing values.

### BNPL Data

Each customer has two unique IDs: `user_id` and `consumer_id`. We chose `consumer_id` as the primary identifier for consistency across all datasets.

For the *consumer's fraud probability* data, we checked for duplicates and removed *99* *(0.28%)* entries. The same process was applied to the *merchant's fraud probability* and transactions data, with no duplicates found.

In the merchant's information data, we used regex to split the tags feature into three categories: category, revenue level, and take rate. No duplicates were found.

The consumer's information data had one column encapsulating details like name, address (state and postcode), gender, and consumer ID. We used regex to split this data into separate columns and found no duplicates.

For the transactions data, covering purchases from February 28, 2021, to August 31, 2022, we ensured all transactions fell within this date range, resulting in the removal of *1,651,235 (11.63%)* rows. The same check was applied to the fraud probability data, with no change in the merchant data and *18 (0.05%)* entries removed from the consumer data.

## Preliminary Analysis

In this notebook, we built upon the initially cleaned data by further refining the process and ensuring that key features were consistent with expectations. Exploratory analysis was conducted to check distributions of important variables and merged relevant datasets to derive insights.

#### **Merchant Dataset Merging**  

We merged the merchant information with merchant fraud data and found that out of the initial 114 merchants with available fraud probabilities, only 95 had corresponding merchant information. Approximately 16% of the merchant_fp entries were excluded due to missing details such as category, name, revenue level, and take rate.                                                                  

Number of entries before the merge: **114**  
Number of entries after the merge: **95**

#### **Consumer Dataset Merging**   

The same merging process was applied to the consumer data, but no entries were lost. The number of entries remained consistent before and after the merge.

Number of entries before the merge: **34747**  
Number of entries after the merge: **34747**

#### **Summary Statistics**

Take rate: Minimum of 0.1, maximum of 0.7.  
Merchant fraud probability: Minimum of 18.2, maximum of 99.10.  
Consumer fraud probability: Minimum of 8.29, maximum of 99.25.  
Dollar value: Minimum of 0.00000009, maximum of 105,193.    

All features fell within reasonable ranges, except for dollar values below 1, which were filtered out. This filtering resulted in only a 0.79% loss of data.

#### **Identified Gaps in Merchant Fraud Probabilities** 

We observed that only 48 out of 4,026 distinct merchants had corresponding fraud probabilities. This highlighted the need for a dedicated model to predict merchant fraud probabilities and impute missing values to better assess merchant risk.

#### **Transaction Dataset Merging**:   

Finally, we merged the transactions dataset with both consumer and merchant fraud probabilities, followed by merchant information:

Before the merge: 12,444,998 rows.  
After merging with consumer fraud probabilities: 12,444,998 rows.  
After merging with merchant fraud probabilities: 12,444,998 rows.  
After merging with merchant information: 11,933,087 rows.  

![dollar_value_dist](../plots/dollar_value_dist.png)

The **Dollar Value Distribution** is heavily skewed toward lower transaction values, with extreme outliers exceeding $60,000.   
To address this, outliers should be removed or Winsorized, and a log transformation can be applied to normalize the distribution for better model performance.

![cons_fp_dist](../plots/merchant_fp_dist.png)

The **Merchant Fraud Probability Distribution** shows a concentration of fraud probabilities around 30%, with fewer merchants in the higher-risk range (above 60%). 

The presence of both low- and high-fraud probability merchants suggests that this feature is useful for distinguishing between different levels of merchant risk. 
To handle this, it’s essential to focus on these high-risk merchants, ensure enough data for the rare high-probability cases, and address data imbalance for more accurate fraud detection.

![cons_fp_dist](../plots/consumer_fp_dist.png)

The **Consumer Fraud Probability Distribution** is right-skewed, with most consumers exhibiting low fraud probabilities and a long tail of higher-risk consumers. 

The decreasing frequency towards higher fraud probabilities indicates that consumers with high fraud risk are relatively rare but still significant for risk assessment. 
To improve the model's accuracy, data imbalance should be addressed using oversampling or class weighting, high-risk consumer data should be verified for reliability, and a transformation can be applied to reduce skewness.

<table border='1'>
<tr><th>revenue_level</th><th>total_revenue</th></tr>
<tr><td>e</td><td>62249.06185458652</td></tr>
<tr><td>d</td><td>329616.53593941435</td></tr>
<tr><td>c</td><td>8966666.315755598</td></tr>
<tr><td>b</td><td>2.8012155570699804E7</td></tr>
<tr><td>a</td><td>4.782461194533622E7</td></tr>
</table>
  
<table border='1'>
<tr><th>revenue_level</th><th>count</th></tr>
<tr><td>e</td><td>105868</td></tr>
<tr><td>d</td><td>120391</td></tr>
<tr><td>c</td><td>2906492</td></tr>
<tr><td>b</td><td>3443556</td></tr>
<tr><td>a</td><td>5356780</td></tr>
</table>
  
<table border='1'>
<tr><th>revenue_level</th><th>merchant_avg_fp</th></tr>
<tr><td>e</td><td>69.0953172336121</td></tr>
<tr><td>d</td><td>63.37734364737916</td></tr>
<tr><td>c</td><td>29.80498907688786</td></tr>
<tr><td>b</td><td>31.613147904540234</td></tr>
<tr><td>a</td><td>29.48776610966251</td></tr>
</table>

The data shows that merchants in **revenue level 'a'** contribute the highest total revenue (over 4.78 billion) while maintaining a relatively low average fraud probability (around 29%). 
In contrast, lower-revenue merchants **(levels 'e' and 'd')** pose a much higher fraud risk, with fraud probabilities of 69% and 63%, respectively, despite contributing less to overall revenue.

This suggests that merchants in **revenue level 'a'** are more promising to explore further, as they offer higher potential profits with lower average risk compared to lower-revenue merchants.

<table border='1'>
<tr><th>merchant_abn</th><th>total_commission</th><th>total_revenue</th><th>average_order_value</th><th>total_orders</th><th>merchant_avg_fp</th><th>consumer_avg_fp</th></tr>
<tr><td>79827781481</td><td>589798.056611674</td><td>8648065.34621223</td><td>2036.2762764803933</td><td>4247</td><td>29.694380203418124</td><td>12.773830702266268</td></tr>
<tr><td>48534649627</td><td>551195.6525381529</td><td>8301139.345454112</td><td>142.7907344190954</td><td>58135</td><td>28.576726038839904</td><td>14.923884163361897</td></tr>
<tr><td>32361057556</td><td>550542.205262667</td><td>8328928.975229457</td><td>109.93979560487146</td><td>75759</td><td>NULL</td><td>15.350992988663366</td></tr>
<tr><td>86578477987</td><td>542066.8802185244</td><td>8430278.074938169</td><td>35.039727318190835</td><td>240592</td><td>NULL</td><td>14.76958039357884</td></tr>
<tr><td>38700038932</td><td>533606.612570049</td><td>8456523.178606164</td><td>1337.2111288118538</td><td>6324</td><td>NULL</td><td>11.164228026866114</td></tr>
<tr><td>45629217853</td><td>518286.79693797533</td><td>7425312.277048358</td><td>37.880574214991185</td><td>196019</td><td>NULL</td><td>15.028875450084216</td></tr>
<tr><td>96680767841</td><td>512131.79186661786</td><td>8665512.552734649</td><td>315.10954737216906</td><td>27500</td><td>29.55524469042502</td><td>14.68158473092625</td></tr>
<tr><td>21439773999</td><td>507995.8506295756</td><td>8327800.829993041</td><td>78.16155303805907</td><td>106546</td><td>28.5044790481046</td><td>14.668721361865119</td></tr>
<tr><td>63123845164</td><td>498439.69520628126</td><td>7563576.558517164</td><td>751.6224345142764</td><td>10063</td><td>NULL</td><td>11.570886106200383</td></tr>
<tr><td>64403598239</td><td>494173.5298797193</td><td>7831593.183513778</td><td>78.11985100910493</td><td>100251</td><td>NULL</td><td>14.59520264877192</td></tr>
</table>


**Profitability Across Order Values**

Merchants can generate substantial revenue with both high and low average order values. Some merchants achieve significant revenue with high-value transactions (over $2,000), while others rely on high volumes of low-value orders (around $35). Order volume could be an important indicator of performance, as it reflects stable and consistent earnings potential across different transaction sizes.

**Revenue vs Total Commission**   
The differences between top merchants by revenue and commission show that take_rate is crucial in evaluating a merchant's value to the BNPL firm, offering a more accurate measure of profitability than revenue alone.

**Fraud Probability:**

The consumer fraud probability across merchants shows relatively low averages, mostly around 12-15%. This indicates that fraud risk on the consumer side is more moderate compared to merchant fraud probabilities, but the risk still exists and may need to be monitored closely, especially for merchants with higher values in both categories. NULL values are still evident, emphasising the need for imputation.


## Geospatial Analysis
We observed that the number of customers varies across different states but remains similar within the same postcode. Therefore, we are interested in analyzing the average fraud probability at both the state and postcode levels to determine if people from different states or postcodes exhibit different scam rates.

We utilized ABS Digital Boundary shapefiles to merge our transaction records with geospatial information at both the postcode and state levels.

![caption](../plots/average_fraud_prob_postcode.png)

![caption](../plots/average_fraud_prob_state.png)

By examining the average fraud probability for each state and postcode, we concluded that using the average fraud probability at the postcode level would be a more effective feature for our fraud detection models. This is because the fraud probability varies significantly across postcodes, ranging from 8% to 53%, whereas at the state level, it only varies between 14.4% and 15.45%.

## Visualisation

### Top Merchant Insights

**Distribution of Revenues** From Revenue level A to D, the top 15 merchants have approximately the same revenue level where as in revenue level E, Et Nunc Consulting seems to make up a large portion of the level's revenue. Click [here](../notebooks/2.3_visualisation.ipynb) to see the pie charts.

### Key Findings from Boxplot Analysis of Revenue Levels

![Boxplot of Revenue Distribution](../plots/boxplot_net_revenue.png) <!-- Replace with the path of your image -->

- **Distribution of Revenues**: The boxplot illustrates the distribution of revenues across different revenue levels, showing a clear separation between lower (`a` and `b`) and higher (`d` and `e`) revenue categories.

- **Presence of Outliers**: A significant number of outliers are observed in the higher revenue levels, indicating that some merchants generate revenues substantially above the median, highlighting extreme revenue disparities.

- **Median vs. Range**: The height of the boxes indicates that the median revenue for higher categories is considerably greater than that of lower categories, reflecting a wider range of revenue generation among higher revenue levels.

- **Mean Deviation**: While not plotted, it can be inferred that the means for higher revenue levels would likely be significantly affected by the outliers, leading to a greater deviation from the median revenue.

- **Implications**: These findings suggest that businesses in higher revenue levels may experience greater variability in performance, with a subset achieving exceptionally high revenues compared to their peers.

### Key Findings from Take Rate Distribution Analysis

![Take Rate Distribution](../plots/take_rate_distribution.png) <!-- Replace with the path of your image -->

- **Take Rate Insights**: The histogram depicts the frequency of various take rate percentages across transactions, providing a visual representation of how often each take rate occurs.

- **Transaction Frequency Trends**: The distribution shows that certain take rate percentages are more common than others, indicating preferred pricing strategies among merchants.




The analysis of take rate distribution and revenue levels reveals significant patterns in transaction behavior and potential risk factors. The frequency distribution of take rates indicates that certain pricing strategies are more commonly adopted, reflecting varied approaches to optimizing revenue. Meanwhile, the revenue level analysis highlights a disparity in merchant performance, with a small subset generating significantly higher revenues. This suggests that some merchants benefit from competitive advantages, but also emphasizes the challenges of forecasting performance across the board. These insights suggest the need for targeted strategies to manage merchant performance and fraud risks, particularly during high-risk periods such as holidays, to enhance profitability and mitigate potential risks.

## Consumer Fraud Probability Model

Since only some customers had predicted fraud probabilities, we used machine learning to estimate missing values. Two approaches were taken:

1. **Consumer-level**: Assigns the same fraud probability to all transactions by a consumer.
2. **Transaction-level**: Assigns different fraud probabilities for each transaction.

We initially expected the second approach to perform better due to more training data and additional features (e.g., order value) that could improve prediction accuracy. In contrast, only *20,128 (4%)* consumers out of 499,999 had fraud probabilities for the first approach.

We engineered features used in both approaches, including average fraud probability, order value, dollar value standard deviation, and transaction count. We also introduced a feature that calculated the percentage of a consumer's median or mean income spent on shopping, assuming those who spend a higher percentage might be more suspicious due to limited funds for necessities like rent and bills.

For the second approach, we added temporal features, such as the purchase month and day of the week. After feature engineering, we applied encoding, standardization, and log-transformation as needed.

For both approaches, we used Linear Regression (LR) as a baseline model and Random Forest Regression (RFR). We evaluated the models using RMSE and $R^2$ metrics.


|                   | RMSE  | R2    |
|-------------------|-------|-------|
| Linear Regression | 8.062 | 0.285 |
| Random Forest     | 7.721 | 0.401 |


For the second approach
|                   | RMSE  | R2    |
|-------------------|-------|-------|
| Linear Regression | 7.830 | 0.241 |
| Random Forest     | 6.811 | 0.426 |

We can see that RFR outperforms LR in both approaches. This is expected as RF is better in capturing nonlinear and complex relationship whereas LR is too simple. As we expected, the RFR model for the second approach is better than the RFR model of the first approach. Thus, we decide to go with the former for the rest of our project.

## Merchant Fraud Probability Model

In this notebook, several new features were engineered to support the development of a model to predict merchant fraud probabilities.   

These features were then used as inputs for both linear regression and random forest regression models, with errors evaluated for each. The best-performing model was selected to generate predictions for ML-based imputation of missing merchant fraud probabilities.

<table border='1'>
<tr><th>merchant_abn</th><th>order_month</th><th>order_datetime</th><th>dollar_value</th><th>order_id</th><th>merchant_fp</th><th>revenue_level</th><th>take_rate</th><th>std_diff_dollar_value</th><th>monthly_order_volume</th><th>avg_monthly_order_volume</th><th>stddev_monthly_order_volume</th><th>std_diff_order_volume</th></tr>
<tr><td>64403598239</td><td>2021-08</td><td>2021-08-27</td><td>62.5219488924545</td><td>c21e53ba-23b2-4943-b3c1-35aa44e1f4a3</td><td>NULL</td><td>a</td><td>6.31</td><td>-0.374873105864042</td><td>5579</td><td>5569.5</td><td>1008.411986885379</td><td>0.009420752751404785</td></tr>
<tr><td>15531408565</td><td>2021-08</td><td>2021-08-14</td><td>184.2471337124171</td><td>f71f6a17-3080-4875-8918-f63e15474c04</td><td>NULL</td><td>a</td><td>6.94</td><td>0.6918129030794428</td><td>57</td><td>66.27777777777777</td><td>15.338554098417012</td><td>-0.6048665159863581</td></tr>
<tr><td>22469567189</td><td>2021-08</td><td>2021-08-21</td><td>334.2561346369639</td><td>0d05f91e-e0ab-4079-950a-40eaa78d36e5</td><td>NULL</td><td>a</td><td>6.99</td><td>-0.7433160715527442</td><td>88</td><td>81.38888888888889</td><td>15.744798912988934</td><td>0.4198917463250146</td></tr>
<tr><td>55430629945</td><td>2021-08</td><td>2021-08-27</td><td>74.46195360174309</td><td>633e0b7d-50f9-4c7d-b4bc-1f2f36fb421e</td><td>NULL</td><td>b</td><td>4.84</td><td>7.400443515785085E-4</td><td>281</td><td>297.22222222222223</td><td>63.460553338789595</td><td>-0.2556268637561117</td></tr>
<tr><td>15061716544</td><td>2021-08</td><td>2021-08-20</td><td>13.479688004962977</td><td>b472b16d-dd25-4018-95d1-89cb0421ae1d</td><td>NULL</td><td>a</td><td>5.73</td><td>-0.9087821440885463</td><td>885</td><td>880.4444444444445</td><td>154.13638812360583</td><td>0.02955535426133334</td></tr>
</table>

**Flagging Unusual Transactions**

The table includes the feature std_diff_dollar_value, which measures how far each transaction deviates from the merchant’s average dollar value, expressed in standard deviations. This feature helps flag transactions that are unusually high or low for each merchant, based on their typical transaction patterns.

**Flagging Unusual Monthly Transaction Volumes**

The table also includes std_diff_order_volume, which shows how much each month's order volume deviates from the merchant’s typical monthly volume. This feature is calculated by comparing the actual monthly order volume to the merchant’s average, using standard deviations to highlight months with unusually high or low activity.

**Additional Date Features**

The table was also adapted to include additional features (not shown above), including the extracted day of the week and flags for transactions occurring on weekends. The year and month columns provide more granular insights for time-based analysis of transaction patterns.

##### *Linear Regression*

Root Mean Squared Error (RMSE) on validation data = 5.253619662832486
R2 (Coefficient of Determination) on validation data: 0.3064055670460758

##### *Random Forest*

Root Mean Squared Error (RMSE) on validation data = 2.4331797228693293
R2 (Coefficient of Determination) on validation data: 0.8512226844236321

#### **Model Performance Summary**  

The linear regression model produced a Root Mean Squared Error (RMSE) of 5.25 and an R² of 0.31 on the validation data. This indicates that while the model explains some of the variance in the data (around 31%), there is still considerable error, suggesting that the linear regression model may struggle to capture more complex patterns in the data.  

In contrast, the random forest regression model significantly outperformed linear regression, with an RMSE of 2.43 and an R² of 0.85. This indicates that the random forest model explains 85% of the variance, demonstrating a much better fit to the data. The lower RMSE also shows that the random forest model makes more accurate predictions compared to the linear regression model.  

**Insights**
The performance comparison suggests that random forest regression is a more suitable choice for predicting merchant fraud probabilities. Its ability to capture non-linear relationships and interactions between features likely contributed to its superior performance, making it the preferred model for generating predictions.

Best Model RMSE on test data = 2.4331797228693293  
Best number of trees: 20  
Best max depth: 5  
Best max bins: 32  

The relatively shallow depth (5) and moderate number of trees (20) suggest that the model effectively captures patterns in the data without overfitting. The number of bins used (32) ensures that sufficient detail is captured when splitting data at each node. These results confirm that the random forest model provides a good balance between complexity and predictive power for estimating merchant fraud probabilities.

<table border='1'>
<tr><th>Feature</th><th>Importance</th></tr>
<tr><td>norm_monthly_order_volume</td><td>0.314107</td></tr>
<tr><td>month_index</td><td>0.151475</td></tr>
<tr><td>norm_std_diff_order_volume</td><td>0.133240</td></tr>
<tr><td>norm_dollar_value</td><td>0.109563</td></tr>
<tr><td>revenue_index</td><td>0.097277</td></tr>
<tr><td>take_rate</td><td>0.094642</td></tr>
<tr><td>weekday_index</td><td>0.060598</td></tr>
<tr><td>norm_std_diff_dollar_value</td><td>0.032338</td></tr>
<tr><td>year_index</td><td>0.004853</td></tr>
<tr><td>is_weekend_vector</td><td>0.001908</td></tr>
</table>

**Feature Importance:**  
The random forest model identified norm_monthly_order_volume as the most important feature, contributing 31.41% to the model's predictive power.   
This indicates that the volume of monthly transactions, normalized across merchants, plays a crucial role in predicting fraud probabilities.

Other key features include:

*month_index: (15.15%)* — the month of the transaction is also highly relevant, indicating that fraud patterns may vary significantly over time.  
*norm_std_diff_order_volume: (13.32%)* — variations in monthly order volume also play a significant role, highlighting that deviations from normal transaction volumes are important fraud indicators.  
*norm_dollar_value: (10.96%)* — the normalized transaction value contributes meaningfully, suggesting that unusually high or low transaction values could signal potential fraud.  

Smaller but still relevant contributors include take_rate, weekday_index, and norm_std_diff_dollar_value, each influencing the model to varying degrees.   
year_index and is_weekend_vector were found to have minimal impact.  

## Ranking System

We adopted the perspective of a Finance Project Manager and treated each merchant as an investment generating revenue for the BNPL firm. We used Discounted Cash Flow (DCF) to estimate a merchant's total revenue by summing future revenue and adjusting for the time-value of money, as money today is worth more than in the future.

$$ \text{DCF} = \sum^{n}_{t=1}\frac{CF_t}{(1+r)^t}$$

The `r` is a discount rate, assumed to be the same for all merchants, based on the Victoria State Government's guidelines. This makes ranking merchants intuitive: we simply select the top 100 with the highest estimated total revenue.

We calculated DCF using forecasted revenues for September, October, and November 2022. The DCF value was then multiplied by the take rate to determine the revenue for the BNPL firm.

To forecast revenues, we used two approaches. The first was calculating the average monthly growth rate for all merchants, but we restricted the range to May 2021–August 2022 to eliminate merchants with no sales in certain months, ensuring balanced growth rates over 15 months.

The second approach used a Long Short-Term Memory (LSTM) model, but the forecasts varied significantly with each run. Fine-tuning and pre-training could fix this, but it was computationally expensive, so we ultimately used the first method.

After calculating the money that goes to the firm, we adjust it by multiplying it with the combined fraud probability of both merchants and consumers, using a weighted average. We assign weights of $\alpha = 0.65$ to the merchant's fraud probability and $\beta = 0.35$ to the consumer's. The formula for the combined fraud probability is:

$$\text{Combined Fraud Probability (CBF)} = \alpha \times \text{Merchant's FP} + \beta\times\text{Consumer's FP}$$

We identified merchants with unrealistically high average monthly revenue growth (marked as red points in the plot), with the highest being 5108%. This prompted us to apply Winsorizing to remove outliers in growth rates.

![avg_growth_rate](../plots/growth_rate_v2.png)


Some merchants have very few orders per month. From a BNPL perspective, we prefer merchants with higher order volumes, as this typically results in more revenue for the BNPL firm. We also found that low average monthly order volumes lead to unstable growth rates and unrealistic revenue forecasts. Therefore, we need a weight that penalizes merchants with low order volumes, which we calculate using a Sigmoid function.

$$ W_{\text{num orders}} = \frac{1}{1 + e^{-(\bar{x_i} - \bar{x_{.}})}}$$

where $\bar{x_i}$ is the average number of order of merchant $i$ and $\bar{x_.}$ is the average number of order of all merchants.

![order_volume](../plots/order_volume_v2.png)

The **coefficient of variation** is a ratio between the standard deviation and the mean, measuring the relative stability which help us compare merchants with different average revenue. Thus, we will create a weight that favors merchant with higher stability. The weight is calculate as

$$W_{\text{CV}} = \frac{1}{1 + CV}$$

## Segmenting Merchants

The merchant dataset was analyzed to group similar business categories into broader segments. 
This categorization helps understand the distribution of merchants across different types of businesses 
and their contribution to various revenue levels.

### Key Segments Identified:
1. **Books, Media, Arts, Crafts, and Hobbies**:
   - Includes categories like bookstores, digital goods, art dealers, and hobby shops.
   - It has a substantial count, indicating a large number of merchants operating in these areas.

2. **Computers, Electronics, and Office Supplies**:
   - Comprises categories such as computer programming services, software shops, and office supplies.
   - This segment captures a significant part of the market, especially merchants dealing with technology.

3. **Home, Garden, and Furnishings**:
   - Contains businesses like furniture shops, garden supply stores, and home improvement outlets.
   - It represents the needs of consumers in home-related products and services.

4. **Fashion, Personal Accessories, Health, and Beauty**:
   - Covers a range of merchants including jewelry shops, optical goods stores, beauty salons, and clothing retailers.
   - This segment highlights a focus on lifestyle, fashion, and personal care.

5. **Vehicles, Repairs, and Miscellaneous Services**:
   - Includes categories such as vehicle parts, repair services, and other specialty shops.
   - It serves niche markets and specific consumer needs.

### Total Estimated Value of Segments
![Take Rate Distribution](../plots/total_revenue_segments_v2.png) <!-- Replace with the path of your image -->

This graph shows the estimated value of each segment, with **Books, Media, Crafts, and Hobbies** leading at **$1.42M**. While selecting more merchants from this segment could maximize profit, we chose to distribute merchants equally across segments to reduce reliance on any single one. Ultimately, the decision depends on the firm's strategy, and we assume they prefer the balanced approach.

### Insights:
- The **Books, Media, Arts, Crafts, and Hobbies** segment has the highest estimated value at **$1.42M**, indicating a strong market potential in creative and digital goods sectors. This suggests that there is significant revenue opportunity within this category.
- The **Computers, Electronics, and Office Supplies** segment follows with a notable value, reflecting the ongoing demand for technology-related products and services. This segment also shows potential for revenue growth, especially with an increasing reliance on technology.
- The **Home, Garden, and Furnishings** and **Fashion, Personal Accessories, Health, and Beauty** segments exhibit moderate values, indicating stable markets that can be tapped into, although they do not lead in overall revenue.
- The **Vehicles, Repairs, and Miscellaneous Services** segment has the lowest estimated value, suggesting it may not be as lucrative as other categories. However, it serves niche markets that can be important for targeted strategies.
- The distribution of merchants across these segments is crucial for the BNPL firm's strategy, as diversifying partnerships can reduce risks and dependencies on any single segment, ultimately aiming for a balanced portfolio approach.

The segment classification provides a clearer picture of the types of merchants in the dataset, enabling 
better decision-making for strategic initiatives like targeted marketing or partnership offers.